This notebook adapted from https://gitlab.com/palisade/palisade-python-demo

Visit the original project for bug reports, updates and local installation instructions.

---

<a href="https://colab.research.google.com/github/Textualization/riiaa21_ws11_ml_over_encrypted_data/blob/main/notebooks/3_Palisade_python_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In [5]:
!git clone https://github.com/Textualization/riiaa21_ws11_ml_over_encrypted_data

rm: cannot remove 'sample_data': No such file or directory
Cloning into 'riiaa21_ws11_ml_over_encrypted_data'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 20), reused 45 (delta 14), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [8]:
!cp /content/riiaa21_ws11_ml_over_encrypted_data/dependencies/palisade-install.tar.gz /content; cd /content; tar -xzf palisade-install.tar.gz

In [10]:
!mkdir -p /content/riiaa21_ws11_ml_over_encrypted_data/build/lib
!cd /content/riiaa21_ws11_ml_over_encrypted_data; c++ -fopenmp -fPIC -std=gnu++11 `python3.7-config --cflags` `python3.7-config --includes` -I/content/palisade-install/include/palisade{,/core,/pke,/third-party/inclue} -o build/lib/ckks_wrapper.o -c src/ckks_wrapper.cpp 
!cd /content/riiaa21_ws11_ml_over_encrypted_data; c++ -fopenmp -fPIC -std=gnu++11 `python3.7-config --cflags` `python3.7-config --includes` -I/content/palisade-install/include/palisade{,/core,/pke,/third-party/inclue} -o build/lib/pycrypto.o -c src/pycrypto.cpp 
!cd /content/riiaa21_ws11_ml_over_encrypted_data; /usr/bin/c++ -fPIC  -Wall -Werror -O3  `python3.7-config --ldflags` -DPALISADE_VERSION=1.11.4 -Wno-parentheses -fopenmp -shared -Wl,-soname,pycrypto.so.1 -lpython3.7m -o build/lib/pycrypto.so.1 build/lib/ckks_wrapper.o build/lib/pycrypto.o   -L/content/palisade-install/lib  -Wl,-rpath,/content/palisade-install/lib:  /content/palisade-install/lib/libPALISADEcore.so.1 /content/palisade-install/lib/libPALISADEbinfhe.so.1 /content/palisade-install/lib/libPALISADEpke.so.1  /usr/lib/x86_64-linux-gnu/libpython3.7m.so /usr/lib/x86_64-linux-gnu/libboost_python3-py36.so 
!cd /content/riiaa21_ws11_ml_over_encrypted_data/build/lib; ln -s pycrypto.so.1 pycrypto.so

In [11]:
import sys
sys.path.append('/content/riiaa21_ws11_ml_over_encrypted_data/build/lib')

import pycrypto

In [12]:
import random
import csv

In [13]:
############################################
# computes next smallest power of two >= x

def next_power_of_2(x):
    return 1 if x == 0 else 2**(x - 1).bit_length()

############################################
# Reads lsvm-model.csv file and returns:
# beta - scaled beta, i.e. beta/s
# bias - scaled bias, i.e. bias/s
# feature_count - beta length

def read_model_data(model_csv):
    csv_file = open(model_csv)
    csv_reader = csv.reader(csv_file, delimiter=",")
    feature_count = 0
    beta = []
    for row in csv_reader:
        if feature_count == 0:
            s = float(row[0])
            print('s ', s)
        else:
            beta.append(float(row[0]))
        feature_count += 1
    feature_count = feature_count - 2
    bias = beta[feature_count:(feature_count + 1)]
    beta = beta[0:feature_count]
    beta[:] = [item / s for item in beta]
    return beta, bias, feature_count

############################################
# Reads lsvm-model.csv file and returns:
# beta - scaled beta, i.e. beta/s
# bias - scaled bias, i.e. bias/s
# feature_count - beta length
# mu - data normalization parameter
# sigma - data normalization parameter


def read_model_data_unnorm(model_csv):
    csv_file = open(model_csv)
    csv_reader = csv.reader(csv_file, delimiter=",")
    feature_count = 0
    beta = []
    mu = []
    sigma = []
    for row in csv_reader:
        if feature_count == 0:
            s = float(row[0])
            print('s ', s)
        else:
            beta.append(float(row[0]))
            mu.append(float(row[1]))
            sigma.append(float(row[2]))
        feature_count += 1
    feature_count = feature_count - 2
    bias = beta[feature_count:(feature_count + 1)]
    beta = beta[0:feature_count]
    beta[:] = [item / s for item in beta]
    return beta, bias, feature_count, mu, sigma

############################################
# Reads lsvm-input.csv file and outputs:
# x - list of input vectors
# input_count - x length

def read_input_data(input_csv):
    csv_file = open(input_csv)
    csv_reader = csv.reader(csv_file, delimiter=",")
    input_count = 0
    x = []
    for row in csv_reader:
        xitem = []
        for column in row:
            xitem.append(float(column))
        x.append(xitem);
        input_count += 1
    return x, input_count

############################################
# Reads lsvm-input.csv file together with
# normalization parameters and outputs:
# x - list of input vectors
# input_count - x length

def read_input_data_unnorm(input_csv, mu, sigma):
    csv_file = open(input_csv)
    csv_reader = csv.reader(csv_file, delimiter=",")
    input_count = 0
    x = []
    for row in csv_reader:
        xitem = []
        colcount = 0
        for column in row:
            xitem.append(
                (float(column) - float(mu[colcount])) / float(sigma[colcount])
            )
            colcount += 1
        x.append(xitem);
        input_count += 1
    return x, input_count

############################################
# Reads lsvm-check.csv file and outputs:
# check - check list of +1/-1
# check_count - check length

def read_check_data(check_csv):
    csv_file = open(check_csv)
    csv_reader = csv.reader(csv_file, delimiter=",")
    check_count = 0
    check = []
    for row in csv_reader:
        check.append(float(row[0]));
        check_count += 1
    return check, check_count

############################################
# Shuffles the input and check lists
# This function is needed if we test random sublists
def shuffle_data(x, check):
    c = list(zip(x, check))
    random.shuffle(c)
    x, check = zip(*c)
    return x, check


############################################
# Plaintext version of lsvm
# num - number of inputs to be tested
# Outputs prediction list

def lsvm_plain_beta_plain_input(beta, bias, x, num):
    res = []
    for i in range(num):
        betaxi = [a * b for a, b in zip(beta, x[i])]
        ip = sum(betaxi)
        ip = ip + bias[0]
        res.append(ip)
    return res


############################################
# Encrypt the input to the lsvm
# num - number of inputs to be enrypted

def enc_input(crypto, x, num):
    enc_x = []
    for i in range(num):
        enc_x.append(crypto.Encrypt(x[i]))
    return enc_x


############################################
# Encrypted version of lsvm with
# encrypted beta and bias and
# unencrypted input
# num - number of inputs to be tested
# Outputs encrypted prediction list

def lsvm_enc_beta_plain_input(crypto, enc_beta, enc_bias, x, num):
    enc_res = []
    for i in range(num):
        enc_betaxi = crypto.EvalMultConst(enc_beta, x[i])
        enc_ip = crypto.EvalSum(enc_betaxi, next_power_of_2(feature_count))
        enc_svm = crypto.EvalAdd(enc_ip, enc_bias)
        enc_res.append(enc_svm)
    return enc_res

############################################
# Encrypted version of lsvm with
# encrypted beta and bias and
# encrypted input
# num - number of inputs to be tested
# Outputs encrypted prediction list

def lsvm_enc_beta_enc_input(crypto, enc_beta, enc_bias, enc_x, num):
    enc_res = []
    for i in range(num):
        enc_betaxi = crypto.EvalMult(enc_beta, enc_x[i])
        enc_ip = crypto.EvalSum(enc_betaxi, next_power_of_2(feature_count))
        enc_svm = crypto.EvalAdd(enc_ip, enc_bias)
        enc_res.append(enc_svm)
    return enc_res

############################################
# Decrypt the output of the LSVM
# num - number of outputs to be decrypted

def dec_output(crypto, enc_res, num):
    res = []
    for i in range(num):
        dec_res = crypto.Decrypt(enc_res[i])
        res.append(dec_res[0])
    return res

############################################
# Timing utilities
############################################
# check the default timer and return the start time in uSec

def tic():
    import timeit
    start_time = timeit.default_timer()
    return start_time

############################################
# check the default timer return elapsed time from start_t

def toc(start_time):
    import timeit
    elapsed = timeit.default_timer() - start_time
    return elapsed

############################################
# same as toc except the result is printed and no value returned


def print_toc(start_time, printstring, units="msec"):
    import timeit
    elapsed = timeit.default_timer() - start_time
    if units == "msec":
        elapsed *= 1000.0
        print(printstring, " {0:5.3f} ms".format(elapsed))
        return elapsed
    else:
        print("unknown units")
        

In [17]:
############################################
# Main Program
############################################

model = "credit"
verbose = True
num_test = 1

print("verbose ", verbose)
print("model ", model)

# credit and  ovarian models are stored unnormalized
model_fn = "/content/riiaa21_ws11_ml_over_encrypted_data/data/lsvm-" + model + "-model.csv"
input_fn = "/content/riiaa21_ws11_ml_over_encrypted_data/data/lsvm-" + model + "-input.csv"
check_fn = "/content/riiaa21_ws11_ml_over_encrypted_data/data/lsvm-" + model + "-check.csv"
if (model == "credit" or model == "ovarian"):
    beta, bias, feature_count, mu, sigma = read_model_data_unnorm(model_fn)
    x, input_count=read_input_data_unnorm(input_fn, mu, sigma)
    check, check_count=read_check_data(check_fn)
else:
    beta, bias, feature_count=read_model_data(model_fn)
    x, input_count=read_input_data(input_fn)
    check, check_count=read_check_data(check_fn)

verbose  True
model  credit
s  0.563246


In [18]:
print("feature_count:", feature_count)
print("input_count:", input_count)
print("check_count:", check_count)

if num_test > input_count:
    num_test=input_count

if num_test == -1:
    num_test=input_count

print("number to test:", num_test)

# CKKS related parameters
max_depth=1
scale_factor=50
batch_size=next_power_of_2(feature_count + 1)

print("-----Initializing ckks wrapper-----")
st=tic()
crypto=pycrypto.CKKSwrapper()
print_toc(st, "Initialized wrapper")


feature_count: 6
input_count: 3932
check_count: 3932
number to test: 1
-----Initializing ckks wrapper-----
Initialized wrapper  0.066 ms


0.06628000005548529

In [19]:
st=tic()
crypto.KeyGen(max_depth, scale_factor, batch_size)
print_toc(st, "Keys generated")

Keys generated  95.919 ms


95.91888800002835

In [20]:
st=tic()
enc_beta=crypto.Encrypt(beta)
print_toc(st, "Betas encrypted")
st=tic()
enc_bias=crypto.Encrypt(bias)
print_toc(st, "Bias encrypted")

x, check=shuffle_data(x, check)
print("input shuffled")

print_num=10



print("-----START LSVM-----")
print("\nPlaintext version")
st=tic()
res_plain=lsvm_plain_beta_plain_input(beta, bias, x, num_test)
plain_time = print_toc(st, "Plaintext LSVM runtime")
if verbose:
    print("result for plaintext case:      ",
          [round(i,2) for i in res_plain[0:print_num]], "..."
    )

print("\nEncrypted model, Plaintext input data")
st=tic()
enc_res_plain_input=lsvm_enc_beta_plain_input(crypto,
                                                enc_beta, enc_bias, x, num_test)
res_plain_input=dec_output(crypto, enc_res_plain_input, num_test)
plain_in_time = print_toc(st, "Encrypted model plaintext input LSVM runtime")
slowdown = float(plain_in_time)/float(plain_time)
print("{0:g} x plaintext runtime".format(slowdown))
if verbose:
    print("result for plain input case:",
          [round(i,2) for i in res_plain[0:print_num]], "..."
    )
print("\nEncrypted model, Encrypted input data")
st = tic()
enc_x = enc_input(crypto, x, num_test)
enc_res_enc_input = lsvm_enc_beta_enc_input(crypto,
                                            enc_beta, enc_bias, enc_x, num_test)
res_enc_input = dec_output(crypto, enc_res_enc_input, num_test)
enc_in_time = print_toc(st, "Encrypted model encrypted input LSVM runtime")
slowdown = float(enc_in_time)/float(plain_time)
print("{0:g} x plaintext runtime".format(slowdown))
if verbose:
    print("result for enc input case:  ",
          [round(i,2) for i in res_plain[0:print_num]], "..."
    )


Betas encrypted  8.143 ms
Bias encrypted  9.210 ms
input shuffled
-----START LSVM-----

Plaintext version
Plaintext LSVM runtime  0.079 ms
result for plaintext case:       [-2.8] ...

Encrypted model, Plaintext input data
Encrypted model plaintext input LSVM runtime  60.898 ms
770.652 x plaintext runtime
result for plain input case: [-2.8] ...

Encrypted model, Encrypted input data
Encrypted model encrypted input LSVM runtime  73.027 ms
924.135 x plaintext runtime
result for enc input case:   [-2.8] ...


# Playground

In [21]:
crypto.KeyGen(2, 50, 1)

In [22]:
e1 = crypto.Encrypt( [ 0.0 ])

In [23]:
e2 = crypto.Encrypt( [ 1.1 ] )

In [24]:
e3 = crypto.EvalAdd(e1,e2)

In [25]:
crypto.Decrypt(e3)[0]

1.0999999999490266

In [26]:
m1 = crypto.EvalMult(e2,e3)

In [27]:
m2 = crypto.EvalMult(m1,e3)

RuntimeError: ignored